In [9]:
from glob import glob

import torch
import imageio

In [5]:
img_arr = imageio.imread("beckett.png")
img_arr.shape

/var/folders/cj/3fpctnd52vv37y9gpdgg7c200000gn/T/ipykernel_46892/1994149648.py:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread("beckett.png")


(1324, 1762, 4)

that deprecation warning is annoying, how abaout we try using imageio v3?

In [6]:
import imageio.v3 as iio # just copying from their readme

img_arr = iio.imread("beckett.png")
img_arr.shape

(1324, 1762, 4)

By default, iio returns (H, W, C) but for our purposes, we want (C, H, W). `permute` to the rescue! We rotate the channels to the first position.

In [7]:
img = torch.from_numpy(img_arr)
out = img.permute(2, 0, 1)

As mentioned before, this doesn't change the storage layout - so changing `img_arr` would change `img`.

On the upside, it's super cheap.

We're going to want to group images into batches of size _N_ to use as input to our NNs

To be a bit efficient, we can pre-allocate a batch:

In [8]:
N = 3
batch = torch.zeros(N, 3, 256, 256, dtype=torch.uint8)

In [13]:
# cat image dir copied from https://github.com/deep-learning-with-pytorch/dlwpt-code/
imdir = "cat-images"
files = glob(f"{imdir}/*.png")
for i, f in enumerate(files):
    # [:3] chops the alpha channel if it exists
    img = torch.from_numpy(iio.imread(f)).permute(2, 0, 1)[:3]
    batch[i] = img